# Example Scenario 1: Pull-Based Data Ingestion with Azure AI Search & Custom Retrieval-Augmented Generation (RAG) Pattern

- **Populate Azure AI Search Index**: A pull-based approach is used to create a search index in Azure AI Search. A pull model uses indexers connecting to a supported data source, automatically uploading the data into your index. This is the recommended approach for data sources that are frequently updated.

- **LLM Queries with Knowledge Base Integration**: A custom implementation for Retrieval Augmented Generation (RAG) will be used to chat with an LLM. This approach will be contrasted with an out-of-the box approach using the Azure OpenAI Service REST API.

In [1]:
import os
import dotenv
import sys

# common setup
dotenv.load_dotenv(".env")
sys.path.append(os.path.join(os.getcwd(), ".."))

---

## 1. Populate Azure AI Search Index

### Approach: Pull-Based Custom Client

This approach will use the `CustomSearchClient` class defined in `search/custom_search_client_pull.py`.

In [ ]:
from search.custom_search_client import CustomSearchClient

# Create search client
search_client = CustomSearchClient(
    search_endpoint=os.environ["AZURE_AI_SEARCH_ENDPOINT"],
)

In [ ]:
# Generate list of variables to be used in templates
template_variables = {
    key: value for key, value in os.environ.items() if key.startswith(("AZURE"))
}

# Define template paths
base_path = os.path.join(os.getcwd(), "..", "search", "templates")
datasource_template_path = os.path.join(base_path, "product-info", "datasource.json")
index_template_path = os.path.join(base_path, "product-info", "index.json")
skillset_template_path = os.path.join(base_path, "product-info", "skillset.json")
indexer_template_path = os.path.join(base_path, "product-info", "indexer.json")

# List of search assets
assets = [
    {
        "type": "indexes",
        "name": os.environ["AZURE_AI_SEARCH_INDEX_NAME"],
        "template_path": index_template_path,
        "template_variables": template_variables,
    },
    {
        "type": "datasources",
        "name": os.environ["AZURE_AI_SEARCH_DATASOURCE_NAME"],
        "template_path": datasource_template_path,
        "template_variables": template_variables,
    },
    {
        "type": "skillsets",
        "name": os.environ["AZURE_AI_SEARCH_SKILLSET_NAME"],
        "template_path": skillset_template_path,
        "template_variables": template_variables,
    },
    {
        "type": "indexers",
        "name": os.environ["AZURE_AI_SEARCH_INDEXER_NAME"],
        "template_path": indexer_template_path,
        "template_variables": template_variables,
    },
]

# Load search asset templates
search_client.load_search_management_asset_templates(assets)

In [ ]:
# Create the index
index_response = search_client.create_search_management_asset(asset_type="indexes")

# Create the data source
datasource_response = search_client.create_search_management_asset(
    asset_type="datasources"
)

# Create skillset to enhance the indexer
skillset_response = search_client.create_search_management_asset(asset_type="skillsets")

# Create the indexer
indexer_response = search_client.create_search_management_asset(asset_type="indexers")

In [ ]:
# Run the indexer
indexer_run_response = search_client.run_indexer()

In [ ]:
# [Optional] Run the indexer with reset
# indexer_run_reset_response = search_client.run_indexer(reset_flag=True)

---

## 2. LLM Queries with Knowledge Base Integration


In [1]:
import os
import dotenv
import sys

dotenv.load_dotenv(".env")
sys.path.append(os.path.join(os.getcwd(), ".."))

### Approach 1: Custom Client

This approach will use the `CustomRetrievalAugmentedGenerationClient` class defined in `open_ai/custom_rag_client.py`. This will NOT require a Microsoft managed private endpoint for private access.

In [2]:
import yaml

# Load configuration file
system_prompt_configuration_file = "../open_ai/configuration.yml"
with open(system_prompt_configuration_file, "r", encoding="utf-8") as f:
    configuration = yaml.safe_load(f)

# Get system messages
query_system_message = configuration.get("query_system_message")
chat_system_message = configuration.get("product_info_chat_system_message")

In [3]:
from open_ai.custom_rag_client import CustomRetrievalAugmentedGenerationClient

# Create orchestration client
rag_client = CustomRetrievalAugmentedGenerationClient(
    open_ai_endpoint=os.getenv("AZURE_OPENAI_API_BASE"),
    open_ai_chat_deployment=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT"),
    open_ai_embedding_deployment=os.getenv("AZURE_OPENAI_EMBEDDING_DEPLOYMENT"),
    search_endpoint=os.getenv("AZURE_AI_SEARCH_ENDPOINT"),
    search_index_name=os.getenv("AZURE_AI_SEARCH_INDEX_NAME"),
    query_system_message=query_system_message,
    chat_system_message=chat_system_message,
)

In [4]:
message_history = []
message_history = rag_client.get_answer(
    "Which tent is the most waterproof?", message_history=message_history
)

In [5]:
for message in message_history:
    print(f"{message['role'].title()}: {message['content']}\n")

User: Which tent is the most waterproof?

Assistant: Based on the information provided, the most waterproof tent is the Alpine Explorer Tent [product_info_8.md]. It provides reliable protection against rain and moisture.



In [6]:
message_history = rag_client.get_answer(
    "Tell me more about the Alpine Explorer Tent?", message_history=message_history
)

In [7]:
for message in message_history:
    print(f"{message['role'].title()}: {message['content']}\n")

User: Which tent is the most waterproof?

Assistant: Based on the information provided, the most waterproof tent is the Alpine Explorer Tent [product_info_8.md]. It provides reliable protection against rain and moisture.

User: Tell me more about the Alpine Explorer Tent?

Assistant: The Alpine Explorer Tent is a reliable and convenient tent for camping. Here are some key features and information about the tent [product_info_8.md]:

- Brand: AlpineGear
- Category: Tents
- Price: $350
- Waterproof: Provides reliable protection against rain and moisture.
- Easy Setup: Simple and quick assembly process, thanks to color-coded poles and intuitive design.
- Accommodation: Can accommodate two queen-sized air mattresses.

Additionally, here is some warranty information about the tent:
- Warranty Duration: The Alpine Explorer Tent is covered by a 2-year limited warranty from the date of purchase.
- Warranty Coverage: The warranty covers manufacturing defects in materials and workmanship, includ

### Approach 2: Azure OpenAI Service REST API

This will require public access on Azure AI Search or a Microsoft managed private endpoint for private access.

In [8]:
from azure.identity import DefaultAzureCredential
import requests

credential = DefaultAzureCredential()
access_token = credential.get_token("https://cognitiveservices.azure.com/.default")

open_ai_endpoint = os.getenv("AZURE_OPENAI_API_BASE")
open_ai_chat_deployment = os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT")
open_ai_api_version = os.getenv("AZURE_OPENAI_API_VERSION")

chat_endpoint = f"{open_ai_endpoint}/openai/deployments/{open_ai_chat_deployment}/chat/completions?api-version={open_ai_api_version}"

request_headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {access_token.token}",
}


def get_answer(message_history: list):
    request_payload = {
        "data_sources": [
            {
                "type": "azure_search",
                "parameters": {
                    "endpoint": os.getenv("AZURE_AI_SEARCH_ENDPOINT"),
                    "index_name": os.getenv("AZURE_AI_SEARCH_INDEX_NAME"),
                    "query_type": "vector_semantic_hybrid",
                    "embedding_dependency": {
                        "deployment_name": os.getenv(
                            "AZURE_OPENAI_EMBEDDING_DEPLOYMENT"
                        ),
                        "type": "deployment_name",
                    },
                    "fields_mapping": {"title_field": "title", "url_field": "path"},
                    "authentication": {"type": "system_assigned_managed_identity"},
                },
            }
        ],
        "messages": message_history,
    }

    response = requests.post(
        chat_endpoint,
        headers=request_headers,
        json=request_payload,
    )

    return response.json()

In [9]:
inital_user_message = "Which tent is the most waterproof?"
message_history = [
    {"role": "system", "content": chat_system_message},
    {"role": "user", "content": inital_user_message},
]

response = get_answer(message_history)
message_history.append(
    {"role": "assistant", "content": response["choices"][0]["message"]["content"]}
)

In [ ]:
for message in message_history:
    if message["role"] in ["user", "assistant"]:
        print(f"{message['role'].title()}: {message['content']}\n")

In [ ]:
follow_up_user_message = "Tell me more about the Alpine Explorer Tent?"
message_history.append({"role": "user", "content": follow_up_user_message})

response = get_answer(message_history)
message_history.append(
    {"role": "assistant", "content": response["choices"][0]["message"]["content"]}
)

In [ ]:
for message in message_history:
    if message["role"] in ["user", "assistant"]:
        print(f"{message['role'].title()}: {message['content']}\n")